# Library Manager Test - With Progress Tracking

This notebook tests the LibraryManager with better progress visibility.

## Setup

In [1]:
import sys
from pathlib import Path

project_root = str(Path.cwd().parent.parent.parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"✅ Project root: {project_root}")

✅ Project root: /home/moad/desktop/open-books


## Import LibraryManager

In [2]:
from src.utils.config import settings, LibreryConfig

print("✅ Imports successful")

✅ Imports successful


In [6]:
"""Library management for syncing PDFs to vector store."""

import hashlib
import json
from pathlib import Path
from typing import Dict

from src.ingestion.chunking.get_chunker import get_chunker
from src.ingestion.embedding.get_embbedder import get_embedder
from src.ingestion.parsers.get_parser import get_parser
from src.ingestion.vector_store.stores import get_store, ChromaStore
from src.utils.logger import logger


class LibraryManager:
    """Manages PDF library syncing with vector store."""

    def __init__(self, config: LibreryConfig) -> None:
        self.books_dir = Path(config.books_paths)
        self.manifest_path = Path(config.manifest_path)

        logger.info("Initializing LibraryManager...")

        # Initialize components (embedder loaded once here)
        self.store = ChromaStore(settings.vector_store)
        self.parser = get_parser()
        self.chunker = get_chunker()

        # Load manifest
        self.manifest = self._load_manifest()
        logger.info(f"Loaded manifest with {len(self.manifest)} entries")

    def _load_manifest(self) -> Dict[str, str]:
        """Load manifest file or create empty one."""
        if self.manifest_path.exists():
            try:
                return json.loads(self.manifest_path.read_text())
            except Exception as e:
                logger.warning(f"Failed to load manifest: {e}. Starting fresh.")
                return {}
        return {}

    def _save_manifest(self) -> None:
        """Save manifest to disk."""
        try:
            self.manifest_path.parent.mkdir(parents=True, exist_ok=True)
            self.manifest_path.write_text(json.dumps(self.manifest, indent=2))
            logger.debug("Manifest saved")
        except Exception as e:
            logger.error(f"Failed to save manifest: {e}")

    def _calculate_hash(self, file_path: Path) -> str:
        """Calculate SHA256 hash of file."""
        sha256 = hashlib.sha256()
        with open(file_path, "rb") as f:
            while chunk := f.read(8192):
                sha256.update(chunk)
        return sha256.hexdigest()

    def sync(self) -> None:
        """Sync books directory with vector store."""
        logger.info(f"Starting sync from: {self.books_dir}")

        # Get current PDF files
        current_files = list(self.books_dir.glob("*.pdf"))
        logger.info(f"Found {len(current_files)} PDF files")

        if not current_files:
            logger.warning("No PDF files found in books directory")
            return

        found_filenames = {f.name for f in current_files}

        self._cleanup_deleted_files(found_filenames)

        self._process_files(current_files)

        self._save_manifest()

        logger.success(f"Sync complete! Vector store has {self.store.count()} chunks")

    def _cleanup_deleted_files(self, found_filenames: set) -> None:
        for filename in list(self.manifest.keys()):
            if filename not in found_filenames:
                logger.info(f"File removed: {filename}")
                try:
                    self.store.delete_by_filename(filename)
                    del self.manifest[filename]
                    logger.info(f"Cleaned up {filename} from index")
                except Exception as e:
                    logger.error(f"Failed to clean up {filename}: {e}")

    def _process_files(self, current_files: list) -> None:
        total = len(current_files)

        for idx, file_path in enumerate(current_files, 1):
            name = file_path.name
            logger.info(f"\n[{idx}/{total}] Processing: {name}")

            try:
                # Calculate current hash
                logger.debug(f"Calculating hash for {name}...")
                current_hash = self._calculate_hash(file_path)

                # Check if file changed
                if self.manifest.get(name) == current_hash:
                    logger.info(f"Skipping {name} (unchanged)")
                    continue

                # File is new or changed
                if name in self.manifest:
                    logger.info(f"Content changed: {name}")
                    self.store.delete_by_filename(name)
                else:
                    logger.info(f"New file: {name}")

                # Process file
                self._index_file(file_path, name, current_hash)

            except Exception as e:
                logger.error(f"Failed to process {name}: {e}")
                logger.exception("Full traceback:")
                # Continue with next file instead of crashing

    def _index_file(self, file_path: Path, name: str, file_hash: str) -> None:
        # Parse
        logger.info(f"  Parsing {name}...")
        parsed_doc = self.parser.parse(file_path)
        logger.info(f"  Parsed {parsed_doc.metadata.nbr_pages} pages")

        # Chunk
        logger.info(f"  Chunking {name}...")
        chunked_doc = self.chunker.chunk(parsed_doc)
        logger.info(f"  Created {len(chunked_doc)} chunks")

        

        # Ingest
        logger.info(f"  Storing {name}...")
        self.store.ingest(chunks=chunked_doc)
        logger.info(f"  Stored in vector DB")

        # Update manifest
        self.manifest[name] = file_hash
        logger.success(f"Successfully indexed: {name}")

    def get_stats(self) -> Dict[str, int]:
        """Get library statistics."""
        return {
            "indexed_files": len(self.manifest),
            "total_chunks": self.store.count(),
        }

    def force_reindex(self, filename: str) -> None:
        """Force reindex a specific file."""
        logger.info(f"Force reindexing: {filename}")

        file_path = self.books_dir / filename
        if not file_path.exists():
            logger.error(f"File not found: {filename}")
            return

        # Remove from index
        if filename in self.manifest:
            self.store.delete_by_filename(filename)
            del self.manifest[filename]

        # Reindex
        current_hash = self._calculate_hash(file_path)
        self._index_file(file_path, filename, current_hash)
        self._save_manifest()

    def clear_all(self) -> None:
        """Clear all indexed data."""
        logger.warning("Clearing all indexed data...")
        self.store.clear()
        self.manifest = {}
        self._save_manifest()
        logger.success("All data cleared")

## Check Configuration

In [7]:
# Verify config
config = settings.librery

print(f"📁 Books directory: {config.books_paths}")
print(f"📄 Manifest path: {config.manifest_path}")

# Check if directory exists
books_path = Path(config.books_paths)
if books_path.exists():
    pdf_files = list(books_path.glob("*.pdf"))
    print(f"✅ Found {len(pdf_files)} PDF files:")
    for f in pdf_files:
        print(f"   - {f.name}")
else:
    print(f"❌ Directory not found: {books_path}")

📁 Books directory: /home/moad/desktop/open-books/books
📄 Manifest path: /home/moad/desktop/open-books/config/manifest.json
✅ Found 2 PDF files:
   - 1912_lora_low_rank_adaptation_of_la.pdf
   - Adam A Method for Stochastic Optimization.pdf


## Initialize LibraryManager

In [8]:
print("Initializing LibraryManager...\n")

manager = LibraryManager(config)

print("\n✅ LibraryManager initialized!")

2026-01-31 20:35:33 | INFO     | __main__:__init__:22 - Initializing LibraryManager...
2026-01-31 20:35:33 | INFO     | src.ingestion.vector_store.stores:__init__:25 - creating or getting the collection
2026-01-31 20:35:33 | INFO     | src.ingestion.vector_store.stores:__init__:31 - getting the embedder
2026-01-31 20:35:33 | INFO     | __main__:__init__:31 - Loaded manifest with 2 entries


Initializing LibraryManager...


✅ LibraryManager initialized!


## Check Current Stats

In [9]:
stats = manager.get_stats()

print("📊 Current Stats:")
print(f"   Indexed files: {stats['indexed_files']}")
print(f"   Total chunks: {stats['total_chunks']}")

📊 Current Stats:
   Indexed files: 2
   Total chunks: 304


## Option 1: Sync Library (Incremental)

This will:
- Skip unchanged files
- Index new files
- Re-index changed files
- Remove deleted files

In [10]:
print("Starting incremental sync...\n")
print("=" * 60)

manager.sync()

print("=" * 60)
print("\n✅ Sync complete!")

# Show updated stats
stats = manager.get_stats()
print(f"\n📊 Updated Stats:")
print(f"   Indexed files: {stats['indexed_files']}")
print(f"   Total chunks: {stats['total_chunks']}")

2026-01-31 20:35:33 | INFO     | __main__:sync:62 - Starting sync from: /home/moad/desktop/open-books/books
2026-01-31 20:35:33 | INFO     | __main__:sync:66 - Found 2 PDF files
2026-01-31 20:35:33 | INFO     | __main__:_process_files:98 - 
[1/2] Processing: 1912_lora_low_rank_adaptation_of_la.pdf
2026-01-31 20:35:33 | INFO     | __main__:_process_files:107 - Skipping 1912_lora_low_rank_adaptation_of_la.pdf (unchanged)
2026-01-31 20:35:33 | INFO     | __main__:_process_files:98 - 
[2/2] Processing: Adam A Method for Stochastic Optimization.pdf
2026-01-31 20:35:33 | INFO     | __main__:_process_files:107 - Skipping Adam A Method for Stochastic Optimization.pdf (unchanged)
2026-01-31 20:35:33 | SUCCESS  | __main__:sync:80 - Sync complete! Vector store has 304 chunks


Starting incremental sync...


✅ Sync complete!

📊 Updated Stats:
   Indexed files: 2
   Total chunks: 304


## Option 2: Force Reindex Single File

In [11]:
# Uncomment to force reindex a specific file
# manager.force_reindex("Word2Vec.pdf")

## Option 3: Clear All Data (⚠️ Destructive)

In [12]:
# Uncomment to clear all indexed data
# manager.clear_all()

## Test Search

In [13]:
# Test if indexing worked
test_query = "word embeddings"
results = manager.store.query([test_query], n_result=3)

print(f"🔍 Search results for: '{test_query}'\n")
for i, result in enumerate(results, 1):
    print(f"[{i}] Score: {result.score:.4f}")
    print(f"    File: {result.metadata.source_doc_title}")
    print(f"    Preview: {result.content[:150]}...")
    print()

2026-01-31 20:35:33 | INFO     | src.ingestion.vector_store.stores:query:60 - querying the results
2026-01-31 20:35:33 | INFO     | src.ingestion.vector_store.stores:query:96 - finished the querying - found 3 unique results


🔍 Search results for: 'word embeddings'

[1] Score: 0.7569
    File: BERT Pre-training of Deep Bidirectional Transformers for Language Understanding
    Preview: ## 2.1 Unsupervised Feature-based Approaches

Learning widely applicable representations of words has been an active area of research for decades, inc...

[2] Score: 0.8421
    File: BERT Pre-training of Deep Bidirectional Transformers for Language Understanding
    Preview: arned in translation: Contextualized word vectors. In NIPS .
- Oren Melamud, Jacob Goldberger, and Ido Dagan. 2016. context2vec: Learning generic cont...

[3] Score: 1.0083
    File: BERT Pre-training of Deep Bidirectional Transformers for Language Understanding
    Preview: ## A.2 Pre-training Procedure

To generate each training input sequence, we sample two spans of text from the corpus, which we refer to as 'sentences'...



## Debug: View Manifest

In [14]:
import json

print("📋 Current Manifest:")
print(json.dumps(manager.manifest, indent=2))

📋 Current Manifest:
{
  "1912_lora_low_rank_adaptation_of_la.pdf": "6154f901b7873cfc910e5d57156907f7b776f83f0c5f532a6b721bc614f0be35",
  "Adam A Method for Stochastic Optimization.pdf": "eab9c73ae2ceda884b94830bda99312254bac4806f6c9f045cbab90721ecda31"
}
